## Tool Calling

In this section, we'll explore how to enhance your applications with tool calling capabilities. We'll cover:
1. Setting up and using the Brave Search API
2. Creating custom tools
3. Configuring tool prompts and safety settings

In [22]:
import asyncio
import os
from typing import Dict, List, Optional
from dotenv import load_dotenv

from llama_stack_client import LlamaStackClient
from llama_stack_client.lib.agents.agent import Agent
from llama_stack_client.lib.agents.event_logger import EventLogger
from llama_stack_client.types.agent_create_params import (
    AgentConfig,
    AgentConfigToolSearchToolDefinition,
)

# Load environment variables
load_dotenv()

# Helper function to create an agent with tools
async def create_tool_agent(
    client: LlamaStackClient,
    tools: List[Dict],
    instructions: str = "You are a helpful assistant",
    model: str = "Llama3.1-8B-Instruct",
) -> Agent:
    """Create an agent with specified tools."""
    agent_config = AgentConfig(
        model=model,
        instructions=instructions,
        sampling_params={
            "strategy": "greedy",
            "temperature": 1.0,
            "top_p": 0.9,
        },
        tools=tools,
        tool_choice="auto",
        tool_prompt_format="json",
        enable_session_persistence=True,
    )

    return Agent(client, agent_config)

First, create a `.env` file in your notebook directory with your Brave Search API key:

```
BRAVE_SEARCH_API_KEY=your_key_here
```


In [19]:
async def create_search_agent(client: LlamaStackClient) -> Agent:
    """Create an agent with Brave Search capability."""
    search_tool = AgentConfigToolSearchToolDefinition(
        type="brave_search",
        engine="brave",
        api_key="dummy_value"#os.getenv("BRAVE_SEARCH_API_KEY"),
    )

    return await create_tool_agent(
        client=client,
        tools=[search_tool],
        instructions="""
        You are a research assistant that can search the web.
        Always cite your sources with URLs when providing information.
        Format your responses as:

        FINDINGS:
        [Your summary here]

        SOURCES:
        - [Source title](URL)
        """
    )

# Example usage
async def search_example():
    client = LlamaStackClient(base_url="http://localhost:5001")
    agent = await create_search_agent(client)

    # Create a session
    session_id = agent.create_session("search-session")

    # Example queries
    queries = [
        "What are the latest developments in quantum computing?",
        #"Who won the most recent Super Bowl?",
    ]

    for query in queries:
        print(f"\nQuery: {query}")
        print("-" * 50)

        response = agent.create_turn(
            messages=[{"role": "user", "content": query}],
            session_id=session_id,
        )

        async for log in EventLogger().log(response):
            log.print()

# Run the example (in Jupyter, use asyncio.run())
await search_example()


Query: What are the latest developments in quantum computing?
--------------------------------------------------
inference> FINDINGS:
The latest developments in quantum computing include advancements in quantum processors, quantum algorithms, and quantum error correction. Researchers have made significant progress in developing more powerful and reliable quantum computers, with some companies already showcasing 100-qubit and 127-qubit quantum processors (IBM, 2022; Google, 2022). These advancements have led to breakthroughs in quantum simulations, machine learning, and optimization problems (Bharti, 2022; Zhang, 2022). Additionally, there have been significant improvements in quantum error correction, which is essential for large-scale quantum computing (Gottesman, 2022).

SOURCES:
- IBM Quantum: "Quantum Processors" (https://www.ibm.com/quantum/computer/)
- Google Quantum AI Lab: "Quantum Processors" (https://quantumai.google/alphabet/sub-page-1/)
- Bharti, K. (2022). "Quantum Comput

## 3. Custom Tool Creation

Let's create a custom weather tool:

In [27]:
from typing import TypedDict, Optional, Dict, Any
from datetime import datetime
class WeatherTool:
    """Example custom tool for weather information."""
    
    def get_name(self) -> str:
        return "get_weather"
    
    def get_description(self) -> str:
        return "Get weather information for a location"
        
    def get_params_definition(self) -> Dict[str, ToolParamDefinitionParam]:
        return {
            "location": ToolParamDefinitionParam(
                param_type="str",
                description="City or location name",
                required=True
            ),
            "date": ToolParamDefinitionParam(
                param_type="str",
                description="Optional date (YYYY-MM-DD)",
                required=False
            )
        }
    
    async def run_impl(self, location: str, date: Optional[str] = None) -> Dict[str, Any]:
        """Simulate getting weather data (replace with actual API call)."""
        # Mock implementation
        return {
            "temperature": 72.5,
            "conditions": "partly cloudy",
            "humidity": 65.0
        }

async def create_weather_agent(client: LlamaStackClient) -> Agent:
    """Create an agent with weather tool capability."""
    agent_config = AgentConfig(
        model="Llama3.1-8B-Instruct",
        instructions="""
        You are a weather assistant that can provide weather information.
        Always specify the location clearly in your responses.
        Include both temperature and conditions in your summaries.
        """,
        sampling_params={
            "strategy": "greedy",
            "temperature": 1.0,
            "top_p": 0.9,
        },
        tools=[
            {
                "function_name": "get_weather",
                "description": "Get weather information for a location",
                "parameters": {
                    "location": {
                        "param_type": "str",
                        "description": "City or location name",
                        "required": True,
                    },
                    "date": {
                        "param_type": "str",
                        "description": "Optional date (YYYY-MM-DD)",
                        "required": False,
                    },
                },
                "type": "function_call",
            }
        ],
        tool_choice="auto",
        tool_prompt_format="json",
        input_shields=[],
        output_shields=[],
        enable_session_persistence=True
    )
    
    # Create the agent with the tool
    weather_tool = WeatherTool()
    agent = Agent(
        client=client,
        agent_config=agent_config,
        custom_tools=[weather_tool]
    )
    
    return agent

# Example usage
async def weather_example():
    client = LlamaStackClient(base_url="http://localhost:5001")
    agent = await create_weather_agent(client)
    session_id = agent.create_session("weather-session")
    
    queries = [
        "What's the weather like in San Francisco?",
        "Tell me the weather in Tokyo tomorrow",
    ]
    
    for query in queries:
        print(f"\nQuery: {query}")
        print("-" * 50)
        
        response = agent.create_turn(
            messages=[{"role": "user", "content": query}],
            session_id=session_id,
        )
        
        async for log in EventLogger().log(response):
            log.print()

# For Jupyter notebooks
import nest_asyncio
nest_asyncio.apply()

# Run the example
await weather_example()


Query: What's the weather like in San Francisco?
--------------------------------------------------
inference> {
    "type": "function",
    "name": "get_weather",
    "parameters": {
        "location": "San Francisco"
    }
}


AttributeError: 'WeatherTool' object has no attribute 'run'